# Histogram matching

## Funciones

In [ ]:
import matplotlib.pyplot as plt
from skimage.util import img_as_ubyte
from skimage import io
from glob import glob
import numpy as np
import random
from PIL import Image
from sklearn.linear_model import LinearRegression
import os

def get_image_list(dir):
    '''
    Reads all the images in the specified directory and returns a list of numpy arrays representing the
    images

    Args:
      dir: The directory that contains the images.

    Returns:
      A list of numpy arrays representing the images.
    '''
    if dir[-1]=='/':
        dir = dir[:-1]
    train_label_path = dir + '/*.*'

    train_label_filenames = glob(train_label_path)
    train_label_filenames.sort()

    print( 'Label images loaded: ' + str( len(train_label_filenames)) )

    # read training images and labels
    train_lbl = [ img_as_ubyte( np.array(io.imread( x, as_gray=True ), dtype='uint8') ) for x in train_label_filenames ]
    return train_lbl

def save_images(imgs, dst_path, name_prefix, fnames, format='.png', convert=''):
    '''
    Save images to disk

    Args:
      imgs: The list of images to be saved.
      dst_path: the destination directory where the images will be saved.
      name_prefix: The prefix of the file name.
      fnames: The filenames of the images to be saved.
      format: The format of the output images. Defaults to .png
      convert: 'L' for greyscale, 'RGB' for color, or '' for nothing.
    '''
    for i, img in enumerate(imgs):
        im = Image.fromarray(img)
        if convert != '':
            im = im.convert(convert)
        im.save( os.path.join(dst_path, fnames[i] + name_prefix + format), quality=100, subsampling=0)

def create_dir(dir):
    '''
    Create a directory if it doesn't exist

    Args:
      dir: The directory where the model will be saved.
    '''
    if not os.path.exists(dir):
        os.makedirs(dir)

## Histogram matching Function

In [ ]:
# apply histogram matching to any image (not mask) from sorce dataset, using target mean histogram
# this histogram matching, match the given cumulative histogram to target cumulative histogram
from sklearn.linear_model import LinearRegression

def histogram_matching(target_imgs, apply_prob):
    '''
    Given a set of images, it will obtain their mean histogram. The number of 0s of this histogram will be predicted
     using Linear regression, with the real number of 1 and 2. It returns a function that apply histogram matching,
     using the calculated histogram. This returned function will apply a random histogram matching to each image with probability
     apply_prob.

    Args:
      target_imgs: the target domain images, from which mean histogram will be obtained (with predicted number of 0s)
      apply_prob: probability of applying the histogram matching

    Returns:
      A function that takes an image as input and returns a modified image or the original image, with
    the given probability.
    '''

    LR = LinearRegression()
    hist_mean,_ = np.histogram(np.array(target_imgs).ravel(), bins=np.arange(256))
    reg = LR.fit(np.reshape([1,2],(-1,1)), np.reshape(hist_mean[1:3],(-1,1))) # use next 2 values to predict using LR
    hist_mean[0] = max(0, float(reg.predict(np.reshape([0,],(-1,1))))) # predict 0 values (due to padding)
    hist_mean = hist_mean / np.array(target_imgs).shape[0] # number of images

    # calculate normalized quantiles
    #tmpl_size = target_imgs[0].size # once 0 value is predicted, the sum of pixels are not the same as the one in the image, so size is no longer useful
    tmpl_size = np.sum(hist_mean)
    tmpl_quantiles = np.cumsum(hist_mean) / tmpl_size

    # based on scikit implementation.
    # source: https://github.com/scikit-image/scikit-image/blob/v0.18.0/skimage/exposure/histogram_matching.py#L22-L70
    def _match_cumulative_cdf(source, tmpl_quantiles):
        src_values, src_unique_indices, src_counts = np.unique(source.ravel(),
                                                            return_inverse=True,
                                                            return_counts=True)

        # calculate normalized quantiles
        # replace number of 0s with lineal regression in order to avoid padding
        if src_values[0] == 0:
            if src_values[:3].tolist() == [0,1,2]:
                reg = LR.fit(np.reshape([1,2],(-1,1)), np.reshape(src_counts[1:3],(-1,1))) # use next 2 values to predict using LR
                pred_0 = max( 0, float(reg.predict(np.reshape([0,],(-1,1)))) ) # predict 0 values (due to padding)
            else:
                # images can be completely black
                pred_0 = 1 if len(src_counts) == 1 else 0 # 1 if completely black, else 0

            src_size = (source.size - src_counts[0]) + pred_0 # more efficient than 'sum(src_counts)'
            src_counts[0] = pred_0 # replace histogram 0s with predictted value
        else:
            src_size = source.size # number of pixels

        src_quantiles = np.cumsum(src_counts) / src_size # normalize
        interp_a_values = np.interp(src_quantiles, tmpl_quantiles, np.arange(len(tmpl_quantiles)))
        if src_values[0] == 0:
            interp_a_values[0] = 0 # we want to keep 0s, (padding)

        return interp_a_values[src_unique_indices].reshape(source.shape)

    def random_histogram_matching(image):
        if random.random() < apply_prob:
            result = _match_cumulative_cdf(image, tmpl_quantiles)
        else:
            result = image
        return result

    return random_histogram_matching


## Apply

In [ ]:
# directory with all datasets
data_path = "./mitochondria/"
# dataset names inside the data_path (all combinations will be computed)
datasets = ['Lucchi++', 'VNC', 'Kasthuri++']
# directory where results are going to be stored
out_dir = "./mitochondria/hist_match/"

In [ ]:
for source in datasets:
    for target in datasets:
        if source == target:
            continue

        print("\n S: {}\tT: {}\n".format(source, target))

        tx = get_image_list(os.path.join(data_path, target ,"train", "x"))
        tx2 = get_image_list(os.path.join(data_path, target ,"test", "x"))
        # train and test may have different sizes, so i concatenate flattened vectors (this does not affect the histogram matching)
        target_trainTest_flat = np.concatenate( (np.array(tx).ravel(), np.array(tx2).ravel()) ) # for histogram matching
        hist_match = histogram_matching(target_trainTest_flat, 1)

        for p in ['train', 'test']:
            in_dir = os.path.join(data_path, source, p )
            for t in ['x', 'y']:
                # Paths to the training images and their corresponding labels
                train_input_path = os.path.join(in_dir, t, '*.*')

                # Read the list of file names
                train_input_filenames = glob(train_input_path)
                train_input_filenames.sort()

                # read training images and labels
                sx = [ img_as_ubyte( np.array(io.imread( x, as_gray=True ), dtype='uint8') ) for x in train_input_filenames ]

                if t == 'x':
                    hm_sx = [hist_match(t) for t in sx]
                else:
                    hm_sx = sx

                train_input_filenames = [os.path.splitext(os.path.basename(x))[0] for x in train_input_filenames]

                out_path = os.path.join(out_dir, source, source+"_s-t_"+target, p, t)
                create_dir(out_path)
                try:
                    save_images(hm_sx, out_path, '', train_input_filenames, format='.png')
                except:
                    save_images(hm_sx, out_path, '', train_input_filenames, format='.png', convert='L')


 S: Lucchi++	T: VNC

Label images loaded: 20
Label images loaded: 20

 S: Lucchi++	T: Kasthuri++

Label images loaded: 85
Label images loaded: 75

 S: VNC	T: Lucchi++

Label images loaded: 165
Label images loaded: 165

 S: VNC	T: Kasthuri++

Label images loaded: 85
Label images loaded: 75

 S: Kasthuri++	T: Lucchi++

Label images loaded: 165
Label images loaded: 165

 S: Kasthuri++	T: VNC

Label images loaded: 20
Label images loaded: 20
